In [1]:
!git clone https://github.com/adikan2k/Final-Project-Group-LexiCore.git
%cd Final-Project-Group-LexiCore

Cloning into 'Final-Project-Group-LexiCore'...
remote: Enumerating objects: 415, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 415 (delta 8), reused 1 (delta 1), pack-reused 384 (from 2)
Receiving objects: 100% (415/415), 319.87 MiB | 16.90 MiB/s, done.
Resolving deltas: 100% (166/166), done.
/content/Final-Project-Group-LexiCore


In [2]:
!git checkout trisha-singh

Updating files: 100% (55/55), done.
Branch 'trisha-singh' set up to track remote branch 'trisha-singh' from 'origin'.
Switched to a new branch 'trisha-singh'


In [3]:
!ls

'500 Samples'  'Final Group Presentation'     notebooks   requirements.txt
 code	       'Final Group Project Report'   Outputs	  simple_demo.py
 data	        LICENSE			      README.md   src


In [4]:
!ls data

embeddings  processed  raw


In [5]:
import pandas as pd

df = pd.read_parquet("data/processed/cleaned_papers.parquet")
print(df.shape)
df.head()

(500, 14)


,paper_id,title,authors,original_abstract,cleaned_text,language,sentences,num_sentences,tokens,processed_text,num_tokens,source,year,venue
0,arxiv_2511.15703v2,"Think Visually, Reason Textually: Vision-Langu...","[Beichen Zhang, Yuhang Zang, Xiaoyi Dong, Yuha...",Abstract reasoning from minimal examples remai...,abstract reasoning from minimal examples remai...,en,[Abstract reasoning from minimal examples rema...,1,"[abstract, reasoning, minimal, example, remain...",abstract reasoning minimal example remain core...,173,arxiv,2025,arXiv
1,arxiv_2511.16613v1,Rate-optimal community detection near the KS t...,"[Jingqiu Ding, Yiding Hua, Kasper Lindberg, Da...",We study community detection in the \emph{symm...,we study community detection in the emphsymmet...,en,[We study community detection in the emphsymme...,1,"[study, community, detection, emphsymmetric, s...",study community detection emphsymmetric stocha...,133,arxiv,2025,arXiv
2,arxiv_2511.20992v1,Dataset Poisoning Attacks on Behavioral Clonin...,"[Akansha Kalra, Soumil Datta, Ethan Gilmore, D...",Behavior Cloning (BC) is a popular framework f...,behavior cloning bc is a popular framework for...,en,[Behavior Cloning BC is a popular framework fo...,1,"[behavior, clone, popular, framework, train, s...",behavior clone popular framework train sequent...,127,arxiv,2025,arXiv
3,arxiv_2511.15120v1,Neural Networks Learn Generic Multi-Index Mode...,"[Bohan Zhang, Zihao Wang, Hengyu Fu, Jason D. ...","In deep learning, a central issue is to unders...","in deep learning, a central issue is to unders...",en,"[In deep learning, a central issue is to under...",1,"[deep, learning, central, issue, understand, n...",deep learning central issue understand neural ...,126,arxiv,2025,arXiv
4,s2orc_e31c6664182777a24b19ef09993ded07d1ed326e,Pyramid: A Layered Model for Nested Named Enti...,"[Jue Wang, Lidan Shou, Ke Chen, Gang Chen]","This paper presents Pyramid, a novel layered m...","this paper presents pyramid, a novel layered m...",en,"[This paper presents Pyramid, a novel layered ...",1,"[paper, present, pyramid, novel, layered, mode...",paper present pyramid novel layered model nest...,90,s2orc,2020,Annual Meeting of the Association for Computat...


In [6]:
!pip install sumy transformers sentence-transformers faiss-cpu rank-bm25 lime --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 108.6 MB/s eta 0:00:00


In [7]:
import os
print("CWD before:", os.getcwd())
%cd /content/Final-Project-Group-LexiCore
!git checkout trisha-singh
!git status

CWD before: /content/Final-Project-Group-LexiCore
/content/Final-Project-Group-LexiCore
Already on 'trisha-singh'
Your branch is up to date with 'origin/trisha-singh'.
On branch trisha-singh
Your branch is up to date with 'origin/trisha-singh'.

nothing to commit, working tree clean


In [8]:
%%writefile code/summarization.py
"""
 Summarization Engine

Provides:
- extractive_summary_textrank
- abstractive_summary_bart
- generate_summaries (1-sentence, 3-sentence, 5-bullet)
"""

from typing import Dict, List

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

from transformers import pipeline



def extractive_summary_textrank(text: str, num_sentences: int = 3) -> str:
    """
    Extractive summary using TextRank (Sumy).

    Args:
        text: input document (abstract)
        num_sentences: how many sentences to keep

    Returns:
        Summary as a single string.
    """
    if not isinstance(text, str) or len(text.strip()) == 0:
        return "No text available."

    try:
        parser = PlaintextParser.from_string(text, Tokenizer("english"))
        summarizer = TextRankSummarizer()
        summary_sents = summarizer(parser.document, num_sentences)
        sentences = [str(s) for s in summary_sents]
        if not sentences:
            return "Summary unavailable (text too short)."
        return " ".join(sentences)
    except Exception as e:
        return f"Extractive summary error: {e}"



_bart_summarizer = None

def _get_bart_pipeline():
    global _bart_summarizer
    if _bart_summarizer is None:
        _bart_summarizer = pipeline(
            "summarization",
            model="facebook/bart-large-cnn"
        )
    return _bart_summarizer


def abstractive_summary_bart(
    text: str,
    max_length: int = 120,
    min_length: int = 30,
) -> str:
    """
    Abstractive summary using BART.

    Args:
        text: input document
        max_length, min_length: summary length control

    Returns:
        Summary string.
    """
    if not isinstance(text, str) or len(text.strip()) == 0:
        return "No text available."

    try:
        summarizer = _get_bart_pipeline()
        truncated = text[:1024]   # BART input limit
        out = summarizer(
            truncated,
            max_length=max_length,
            min_length=min_length,
            do_sample=False,
        )
        return out[0]["summary_text"]
    except Exception as e:
        return f"Abstractive summary error: {e}"



def generate_summaries(text: str) -> Dict[str, object]:
    """
    Generates:
    - one_sentence: short abstractive summary
    - three_sentence: extractive summary (3 sentences)
    - five_bullets: list of bullet strings

    Returns:
        dict with keys: one_sentence, three_sentence, five_bullets
    """
    one_sentence = abstractive_summary_bart(
        text, max_length=40, min_length=15
    )
    three_sentence = extractive_summary_textrank(
        text, num_sentences=3
    )

    long_sum = abstractive_summary_bart(
        text, max_length=160, min_length=40
    )

    raw_bullets = [s.strip() for s in long_sum.split(".") if s.strip()]
    raw_bullets = raw_bullets[:5]
    bullets: List[str] = [f"• {b}" for b in raw_bullets]

    return {
        "one_sentence": one_sentence,
        "three_sentence": three_sentence,
        "five_bullets": bullets,
    }

Writing code/summarization.py


In [11]:
%cd /content/Final-Project-Group-LexiCore
!ls

/content/Final-Project-Group-LexiCore
'500 Samples'  'Final Group Presentation'     notebooks   requirements.txt
 code	       'Final Group Project Report'   Outputs	  simple_demo.py
 data	        LICENSE			      README.md   src


In [12]:
import sys
sys.path.append('/content/Final-Project-Group-LexiCore')

In [13]:
%%writefile code/__init__.py
# Makes the 'code' directory a Python package so we can do: from code import ...

Writing code/__init__.py


In [14]:
!ls code

00_START_HERE.ipynb		   __init__.py
COMPLETE_PIPELINE_FULL_DAY1.ipynb  summarization.py


In [18]:
import sys
import os

# Go to your repo root
%cd /content/Final-Project-Group-LexiCore

# Add the code folder to Python import path
sys.path.append("/content/Final-Project-Group-LexiCore/code")

print("Current sys.path entry added!")

/content/Final-Project-Group-LexiCore
Current sys.path entry added!


In [19]:
!ls /content/Final-Project-Group-LexiCore/code

00_START_HERE.ipynb		   __init__.py
COMPLETE_PIPELINE_FULL_DAY1.ipynb  summarization.py


In [20]:
from summarization import generate_summaries

In [21]:
import pandas as pd


df = pd.read_parquet("data/processed/cleaned_papers.parquet")
print("Rows:", len(df))

sample = df.iloc[0]
abstract = sample.get("original_abstract", sample.get("abstract", ""))

print("Sample Abstract:\n", abstract[:500], "...")

summaries = generate_summaries(abstract)

print("\nGenerated Summaries:")
summaries

Rows: 500
Sample Abstract:
 Abstract reasoning from minimal examples remains a core unsolved problem for frontier foundation models such as GPT-5 and Grok 4. These models still fail to infer structured transformation rules from a handful of examples, which is a key hallmark of human intelligence. The Abstraction and Reasoning Corpus for Artificial General Intelligence (ARC-AGI) provides a rigorous testbed for this capability, demanding conceptual rule induction and transfer to novel tasks. Most existing methods treat ARC-A ...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu



Generated Summaries:


{'one_sentence': 'The Abstraction and Reasoning Corpus for Artificial General Intelligence (ARC-AGI) provides a rigorous testbed for this capability. It demands conceptual rule induction and transfer to novel tasks',
 'three_sentence': 'Extractive summary error: NLTK tokenizers are missing or the language is not supported.\nDownload them by following command: python -c "import nltk; nltk.download(\'punkt\')"\nOriginal error was:\n\n**********************************************************************\n  Resource \x1bpunkt_tab\x1b not found.\n  Please use the NLTK Downloader to obtain the resource:\n\n  \x1b>>> import nltk\n  >>> nltk.download(\'punkt_tab\')\n  \x1b\n  For more information see: https://www.nltk.org/data.html\n\n  Attempted to load \x1btokenizers/punkt_tab/english/\x1b\n\n  Searched in:\n    - \'/root/nltk_data\'\n    - \'/usr/nltk_data\'\n    - \'/usr/share/nltk_data\'\n    - \'/usr/lib/nltk_data\'\n    - \'/usr/share/nltk_data\'\n    - \'/usr/local/share/nltk_data\'\n

In [22]:
%%writefile code/digest_pipeline.py
"""
Digest Pipeline
Creates:
- simple keyword retrieval
- summaries for top papers
- digest JSON output
"""

import os
import json
from typing import List, Dict, Any
import pandas as pd
from summarization import generate_summaries



def load_corpus(path="data/processed/cleaned_papers.parquet") -> pd.DataFrame:
    df = pd.read_parquet(path)
    return df


def simple_retrieval(query: str, df: pd.DataFrame, top_k: int = 5) -> List[int]:
    """
    Returns top_k indices where query appears in title or abstract.
    """
    text = (
        df.get("title", "").fillna("").astype(str)
        + " "
        + df.get("original_abstract", df.get("abstract", "")).fillna("").astype(str)
    )

    mask = text.str.contains(query, case=False, na=False)
    matches = df[mask].index.tolist()

    if len(matches) == 0:
        matches = df.index.tolist()

    return matches[:top_k]



def build_digest(query: str, df: pd.DataFrame, top_k: int = 5) -> Dict[str, Any]:
    indices = simple_retrieval(query, df, top_k)
    output = []

    for rank, idx in enumerate(indices, 1):
        row = df.loc[idx]

        abstract = row.get("original_abstract", row.get("abstract", ""))

        summaries = generate_summaries(str(abstract))

        output.append({
            "rank": rank,
            "paper_id": row.get("paper_id", int(idx)),
            "title": row.get("title"),
            "venue": row.get("venue"),
            "year": row.get("year"),
            "summaries": summaries
        })

    digest = {
        "query": query,
        "num_results": len(output),
        "papers": output
    }

    return digest

def save_digest(digest: Dict[str, Any], output_path: str):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w") as f:
        json.dump(digest, f, indent=2)

Writing code/digest_pipeline.py


In [23]:
from digest_pipeline import load_corpus, build_digest

df = load_corpus()
digest = build_digest("transformer", df, top_k=3)

digest

{'query': 'transformer',
 'num_results': 3,
 'papers': [{'rank': 1,
   'paper_id': 's2orc_6825ca62bde5036fccc06567c3c06093150e14d3',
   'title': 'T-NER: An All-Round Python Library for Transformer-based Named Entity Recognition',
   'venue': 'Conference of the European Chapter of the Association for Computational Linguistics',
   'year': np.int64(2022),
   'summaries': {'one_sentence': 'Language model (LM) pretraining has led to consistent improvements in many NLP downstream tasks, including named entity recognition (NER) We present T-NER (Transformer-based Named',
    'three_sentence': 'Extractive summary error: NLTK tokenizers are missing or the language is not supported.\nDownload them by following command: python -c "import nltk; nltk.download(\'punkt\')"\nOriginal error was:\n\n**********************************************************************\n  Resource \x1bpunkt_tab\x1b not found.\n  Please use the NLTK Downloader to obtain the resource:\n\n  \x1b>>> import nltk\n  >>> nltk

In [28]:
import os
import json
import numpy as np

def save_digest(digest, output_path="Outputs/sample_transformer_digest.json"):
    """
    Save digest dictionary as JSON.
    - Converts numpy/pandas types (int64, float64, arrays) to plain Python types
    - Creates parent folder if needed
    """
    folder = os.path.dirname(output_path)
    if folder:  # only try to create if there is a folder part
        os.makedirs(folder, exist_ok=True)

    def convert(o):
        # numpy / pandas integers -> Python int
        if isinstance(o, (np.integer,)):
            return int(o)
        # numpy / pandas floats -> Python float
        if isinstance(o, (np.floating,)):
            return float(o)
        # numpy arrays -> list
        if isinstance(o, np.ndarray):
            return o.tolist()
        # fallback: string
        return str(o)

    with open(output_path, "w") as f:
        json.dump(digest, f, indent=2, default=convert)

In [29]:
output_path = "Outputs/sample_transformer_digest.json"
save_digest(digest, output_path)

!ls Outputs

digests				sbert_title_embeddings.npy
embedding_metadata.json		scibert_embeddings.npy
sample_transformer_digest.json	vocabulary.json
sbert_abstract_embeddings.npy	word2vec_embeddings.npy


In [30]:
import pandas as pd

def run_digest_query(
    query: str,
    top_k: int = 3,
    parquet_path: str = "data/processed/cleaned_papers.parquet",
):
    """
    Convenience helper:
    - loads the corpus
    - builds a digest for the query
    - pretty-prints basic info
    - returns the digest dict
    """
    print(f"🔎 Running digest for query: '{query}' (top_k={top_k})")
    df = load_corpus(parquet_path)
    digest = build_digest(query, df, top_k=top_k)

    print(f"\nFound {digest['num_results']} papers:\n")
    for paper in digest["papers"]:
        title = paper.get("title", "Untitled")
        year = paper.get("year", "NA")
        one_sentence = paper["summaries"].get("one_sentence") if isinstance(paper["summaries"], dict) else None

        print(f"#{paper['rank']} — {title} ({year})")
        if one_sentence:
            print(f"  1-sentence summary: {one_sentence}")
        print()

    return digest

In [31]:
def digest_to_dataframe(digest):
    """
    Turn digest['papers'] into a pandas DataFrame for quick analysis / export.
    """
    rows = []
    for p in digest["papers"]:
        summaries = p["summaries"]
        if isinstance(summaries, dict):
            one_sentence = summaries.get("one_sentence")
            three_sentence = summaries.get("three_sentence")
        else:
            one_sentence = None
            three_sentence = None

        rows.append({
            "rank": p.get("rank"),
            "paper_id": p.get("paper_id"),
            "title": p.get("title"),
            "venue": p.get("venue"),
            "year": p.get("year"),
            "one_sentence_summary": one_sentence,
            "three_sentence_summary": three_sentence,
        })
    return pd.DataFrame(rows)

In [32]:

test_query = "transformer models"
digest = run_digest_query(test_query, top_k=5)

df_digest = digest_to_dataframe(digest)
df_digest.head()

🔎 Running digest for query: 'transformer models' (top_k=5)

Found 3 papers:

#1 — Controlling changes to attention logits (2025)
  1-sentence summary: Stability of neural network weights is critical when training transformer models. The query and key weights are particularly problematic, as they tend to grow large without any intervention. Applying normalization to

#2 — IntAttention: A Fully Integer Attention Pipeline for Efficient Edge Inference (2025)
  1-sentence summary: IntAttention is the first fully integer, plug-and-play attention pipeline without retraining. Our method achieves up to 3.7x speedup and 61% energy redu.

#3 — RoBERTa-LSTM: A Hybrid Model for Sentiment Analysis With Transformer and Recurrent Neural Network (2022)
  1-sentence summary: Social media is a platform for people to express their feelings, feedback, and opinions. Sentiment analysis plays the role to determine whether the sentiment of the text is positive, negative, neutral



,rank,paper_id,title,venue,year,one_sentence_summary,three_sentence_summary
0,1,arxiv_2511.21377v1,Controlling changes to attention logits,arXiv,2025,Stability of neural network weights is critica...,Extractive summary error: NLTK tokenizers are ...
1,2,arxiv_2511.21513v1,IntAttention: A Fully Integer Attention Pipeli...,arXiv,2025,"IntAttention is the first fully integer, plug-...",Extractive summary error: NLTK tokenizers are ...
2,3,s2orc_05405f589e6751bf2dd6678ba077738ee9555fbf,RoBERTa-LSTM: A Hybrid Model for Sentiment Ana...,IEEE Access,2022,Social media is a platform for people to expre...,Extractive summary error: NLTK tokenizers are ...
